# TOV Right-Hand-Side Evaluator

## Detailed Analysis

This script defines the function `tov_rhs`, which computes the derivatives for the system of differential equations describing a static, spherically symmetric neutron star. It serves as the core "physics engine" called by the ODE integrator in `solve_sequence.py`.

The function couples the macroscopic General Relativistic equations (Tolman-Oppenheimer-Volkoff) with the microscopic properties of matter (Equation of State). Additionally, it includes the Riccati equation required to calculate the tidal deformability of the star.

## Physics and Math

The script solves for the radial evolution of three quantities: mass $m(r)$, pressure $P(r)$, and the tidal variable $y(r)$.

### 1. Structure Equations (TOV)
The macroscopic structure is governed by General Relativity:

$$
\frac{dm}{dr} = 4\pi r^2 \epsilon
$$

$$
\frac{dP}{dr} = - \frac{G (\epsilon + P) (m + 4\pi r^3 P)}{r (r - 2Gm)}
$$

where $\epsilon$ is the energy density and $P$ is the pressure. In the code, factors like $G$ and $4\pi$ are handled via conversion constants (`G_CONV`, `A_CONV`).

### 2. Tidal Deformability
To determine the tidal Love number $k_2$, a perturbation equation (the Riccati equation) is solved alongside the TOV equations:

$$
\frac{dy}{dr} = -\frac{1}{r} \left( y^2 + y F(r) + r^2 Q(r) \right)
$$

where $F(r)$ and $Q(r)$ are metric functions that depend on $P$, $\epsilon$, and the speed of sound $c_s^2$.

### 3. Microphysics (Quark Model)
For Quark stars, the pressure $P$ and energy density $\epsilon$ are analytically related via the chemical potential $\mu$:

$$
P = \frac{3}{4\pi^2}\mu^4 + \frac{3}{\pi^2}\left(\Delta^2 - \frac{m_s^2}{4}\right)\mu^2 - B
$$

The script solves this quadratic equation for $\mu^2$ given $P$, and then computes $\epsilon$.

## Code Walkthrough

### 1. Thermodynamics and Microphysics
The first half of the function determines the energy density $\epsilon$ and the speed of sound squared $c_s^2$ (or its inverse, `dedp`) for a given pressure `P`.

**Quark Logic (CFL):**
If `is_quark` is true, the script unpacks the Bag constant $B$, Gap $\Delta$, and strange mass $m_s$. It converts pressure to geometric units and solves the algebraic relation derived from the thermodynamic potential.

```python
# Solve Quadratic for mu^2
det = coeff_b**2 - 4*coeff_a*coeff_c
mu2 = (-coeff_b + np.sqrt(max(0, det))) / (2*coeff_a)

# Calculate Epsilon
eps_geom = 3.0 * coeff_a * (mu2**2) + coeff_b * mu2 + B_geom
```

**Hadronic Logic:**
If `is_quark` is false, the script handles the crust-core transition.
*   **Core:** It interpolates between two parent models (`fA`, `fB`) using the mixing weight $w$ and scales the result by $\alpha$.
*   **Crust:** If $P < P_{trans}$, the analytic crust functions (SLy model) are used.

### 2. Physical Clamps
To ensure the numerical solution remains stable and causal, limits are applied to the calculated speed of sound.

```python
# Enforce Causality (v <= c)
if cs2_local > 1.0: cs2_local = 1.0 
# Enforce Stability (v > 0)
if cs2_local < 1e-5: cs2_local = 1e-5
```

### 3. Macroscopic Derivatives
The second half of the function implements the General Relativistic equations.

**TOV Equations:**
The gradients for pressure `dP_dr` and mass `dm_dr` are computed using the standard Schwarzschild metric terms.

**Riccati Equation:**
The terms $Q$ and $F$ are computed, which involve the metric component $e^\lambda$ (`exp_lambda`) and the speed of sound `cs2_local`. These are combined to find `dy_dr`.

```python
Q = CONSTANTS['A_CONV'] * CONSTANTS['G_CONV'] * (5.0*epsilon + 9.0*P + (epsilon+P)/cs2_local) * (r**2)
Q -= 6.0 * exp_lambda

dy_dr = -(y_tidal**2 + y_tidal * F + Q) / r
```

The function returns the list of derivatives `[dm_dr, dP_dr, dy_dr]` to be used by the RK45 step.